In [ ]:
import langchain
print(langchain.__version__)

In [ ]:
import openai
print(openai.__version__)

In [ ]:
import sys
#查看python的版本
print(sys.version)

# 1.获取大模型

In [ ]:
#导入 dotenv 库的 load_dotenv 函数，用于加载环境变量文件（.env）中的配置
import dotenv
from langchain_openai import ChatOpenAI
import os
dotenv.load_dotenv() #加载当前目录下的 .env 文件
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
# 创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini") # 默认使用 gpt-3.5-turbo
# 直接提供问题，并调用llm
response = llm.invoke("什么是大模型？")
print(response)


# 2.使用提示词模板

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
# 需要注意的一点是，这里需要指明具体的role，在这里是system和用户
prompt = ChatPromptTemplate.from_messages([
 ("system", "你是世界级的技术文档编写者"),
 ("user", "{input}") # {input}为变量
])
# 我们可以把prompt和具体llm的调用和在一起。
chain = prompt | llm
message = chain.invoke({"input": "大模型中的LangChain是什么?"})
print(message)


# 3. 使用输出解析器

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser
# 初始化模型
llm = ChatOpenAI(model="gpt-4o-mini")
# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
 ("system", "你是世界级的技术文档编写者。"),
 ("user", "{input}")
])
# 使用输出解析器
# output_parser = StrOutputParser()
output_parser = JsonOutputParser()
# 将其添加到上一个链中
# chain = prompt | llm
chain = prompt | llm | output_parser
# 调用它并提出同样的问题。答案是一个字符串，而不是ChatMessage
# chain.invoke({"input": "LangChain是什么?"})
chain.invoke({"input": "LangChain是什么? 用JSON格式回复，问题用question，回答用answer"})


# 4. 使用向量存储

In [ ]:
from langchain_ollama import OllamaLLM

# 使用已下载的 deepseek-r1:7b 模型
llm = OllamaLLM(
    model="deepseek-r1:7b",
    temperature=0.7,
    num_predict=512
)

# 测试中文问题
response = llm.invoke("什么是大语言模型？用中文回答")
print(response)

In [ ]:
# 导入
from langchain_ollama import OllamaLLM

# 创建模型（使用 deepseek-r1:7b）
llm = OllamaLLM(
    model="deepseek-r1:7b",
    temperature=0.7,
    num_predict=512
)

# 测试
question = "请用中文解释 LangChain 是什么，并给出一个简单示例"
answer = llm.invoke(question)
print(answer)

# 流式输出（实时显示）

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="deepseek-r1:7b")

for chunk in llm.stream("请写一首关于春天的诗："):
    print(chunk, end="", flush=True)

# 在 LangChain 中完整使用

## 1. 文档加载与处理

In [24]:
# 安装必要的包
!pip install langchain-community langchain-ollama faiss-cpu

# 导入正确的模块
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaLLM  # 注意：这是正确的导入
from langchain.chains import RetrievalQA

# 1. 加载文档 - 使用完整路径
import os
# 获取当前 notebook 所在目录
current_dir = os.getcwd()
file_path = os.path.join(current_dir, "document.txt")

if not os.path.exists(file_path):
    # 如果文件不存在，创建示例文件
    sample_text = """LangChain 是一个用于开发由语言模型驱动的应用程序的框架。
它提供了一套工具、组件和接口，简化了构建基于LLM的应用过程。
主要功能包括：模型调用、提示模板、链式调用、记忆、索引和代理等。
LangChain 支持多种语言模型，包括 OpenAI、Anthropic、Hugging Face 等。"""

    with open(file_path, "w", encoding="utf-8") as f:
        f.write(sample_text)
    print(f"已创建示例文件: {file_path}")

loader = TextLoader(file_path, encoding="utf-8")
documents = loader.load()
print(f"加载了 {len(documents)} 个文档")

# 2. 分割文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
docs = text_splitter.split_documents(documents)
print(f"分割为 {len(docs)} 个片段")

# 3. 创建本地 embeddings
embeddings = OllamaEmbeddings(model="llama3.2")

# 4. 创建向量数据库
vectorstore = FAISS.from_documents(docs, embeddings)
print("向量数据库创建成功")

# 5. 创建检索链
llm = OllamaLLM(model="llama3.2")  # 使用 OllamaLLM，不是 Ollama
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=False
)

# 6. 提问
result = qa_chain.invoke("文档主要讲了什么？")
print("\n回答：")
print(result["result"])

已创建示例文件: D:\MinJiHao\Project\AI-Notes\LangChain\document.txt
加载了 1 个文档
分割为 1 个片段
向量数据库创建成功

回答：
LangChain 的文档主要介绍了 LangChain框架的功能和特点。它简要介绍了 LangChain 提供的工具、组件和接口，以及其支持多种语言模型的能力等。


## 简化版本

In [25]:
# 1. 创建示例文档
sample_text = """LangChain 是一个用于开发由语言模型驱动的应用程序的框架。
它提供了一套工具、组件和接口，简化了构建基于LLM的应用过程。"""

with open("document.txt", "w", encoding="utf-8") as f:
    f.write(sample_text)

# 2. 加载和处理
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaLLM
from langchain.chains import RetrievalQA

loader = TextLoader("document.txt", encoding="utf-8")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = text_splitter.split_documents(docs)

embeddings = OllamaEmbeddings(model="llama3.2")
vectorstore = FAISS.from_documents(chunks, embeddings)

llm = OllamaLLM(model="llama3.2")
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

result = qa.invoke("LangChain 是什么？")
print(result["result"])

LangChain 是一种用于开发语言模型（LLM）驱动的应用程序的框架。
